***
<font size="6"><center><b> HD-CapsNet: A Hierarchical Deep Capsule Network for Image Classification </b></center></font>
***

**Changing Model Architecture**
- **(Mod-2.4)** From coarse-to-Fine slowly decrease the dimension. i.e. 32D>16D>8D (Coarse>Medium>FINE), While changing the primary capsule dimension ($P_{capsule} = 16D$). Training without $L_{cons}$

# Files and Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras import backend as K

import tensorflow_docs as tfdocs
import tensorflow_docs.plots

# Supporting Libraries:
    #Mathplot lib for ploting graphs
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
    # numpy and pandas
import numpy as np
import pandas as pd
    #system
import os
import sys
import csv
    #import other libraries
import math
import random
from datetime import datetime
from treelib import Tree
    # ML model, Dataset and evalution metrics
sys.path.append('../../') ### adding system parth for src folder
from src import datasets # load datasets
from src import MixUp_add_loss # load datasets
from src import metrics # load hierarchcial metrics
from src import sysenv # load hierarchcial metrics
from src import models # load machine learning models

    ## Tensorflow_docs
import tensorflow_docs as tfdocs
import tensorflow_docs.plots

    # Auto reload local libraries if updated
%load_ext autoreload
%autoreload 2

# System information & GPU growth

In [2]:
systeminfo = sysenv.systeminfo()
print(systeminfo)


• Computer Name = SARUMAN
• Working Directory = D:\knoor\Deep Learning with Python\RunningOnServer\HD-CapsNet_MOD_2\Training_and_Analysis\4_CIFAR-100
• Python Version = 3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]
• TensorFlow Version = 2.8.0
• Keras Version = 2.8.0
• Current Environment = Anaconda Environment Name : D:\knoor\AnacondaEnvironment\py38tf28


In [3]:
gpus = "0,1,2,3,4,5,6,7" ## Selecting Available gpus (Multi-GPUS)
gpus = "7" ## Selecting Available gpus (Single GPU)
gpugrowth = sysenv.gpugrowth(gpus = gpus) ## Limiting GPUS from OS environment
gpugrowth.memory_growth() #GPU memory growth

Following GPUS are selected =  7
Done: GPU PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
1 Physical GPUs, 1 Logical GPUs


## log directory

In [4]:
directory = sysenv.log_dir('MOD_2/4_CIFAR_100/HD_CapsNet/TD_WO_C_loss_P16')

FOLDER CREATED =  ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_WO_C_loss_P16


# Import Dataset : CIFAR-100

In [5]:
dataset = datasets.CIFAR100(version = 'ALL') # importing CIFAR10 Dataset
# dataset = datasets.CIFAR100(version = 'reduce') # importing CIFAR10 Dataset


CIFAR-100 dataset: Training have 50,000 samples and testing have 10,000 samples


# Parameters

In [6]:
train_params = {"n_epochs" : 100,
                "batch_size": 64,
                "lr": 0.001, # Initial learning rate
                "lr_decay": 0.95, # Learning rate decay
                "decay_exe": 9, #learning rate decay execution epoch after
               }
model_params = {"optimizer": tf.keras.optimizers.Adam(train_params['lr']),
                "loss_function": models.MarginLoss(),
               }

## Learning Rate Decay Scheduler

In [7]:
def scheduler(epoch):
    learning_rate_init = train_params["lr"]
    
    if epoch > train_params["decay_exe"]:
        learning_rate_init = train_params["lr"] * (train_params["lr_decay"] ** (epoch-9))
        
    tf.summary.scalar('learning rate', data=learning_rate_init, step=epoch)
        
    return learning_rate_init

# Bottom up Approach

## Loss Function

In [8]:
number_of_classes_c = len(np.unique(np.argmax(dataset['y_train_coarse'], axis=1)))
number_of_classes_m = len(np.unique(np.argmax(dataset['y_train_medium'], axis=1)))
number_of_classes_f = len(np.unique(np.argmax(dataset['y_train_fine'], axis=1)))

## For Dynamic LossWeights
initial_lw = models.initial_lw({"coarse": number_of_classes_c,
                                "medium": number_of_classes_m,
                                "fine": number_of_classes_f})

lossweight = {'coarse_lw' : K.variable(value = initial_lw['coarse'], dtype="float32", name="coarse_lw"),
             'medium_lw' : K.variable(value = initial_lw['medium'], dtype="float32", name="medium_lw"),
             'fine_lw' : K.variable(value = initial_lw['fine'], dtype="float32", name="fine_lw"),
              'decoder_lw' : 0.0
             }

In [9]:
def margin_loss(y_true, y_proba):
    
    present_error_raw = tf.square(tf.maximum(0., 0.9 - y_proba), name="present_error_raw")
    absent_error_raw = tf.square(tf.maximum(0., y_proba - 0.1), name="absent_error_raw")
    L = tf.add(y_true * present_error_raw, 0.5 * (1.0 - y_true) * absent_error_raw,name="L")
    total_marginloss = tf.reduce_sum(L, axis=1, name="margin_loss")
    return total_marginloss

def CustomLoss(y_true_c, y_true_m, y_true_f, y_pred_c, y_pred_m, y_pred_f, LW_C, LW_M, LW_F):
    ML_c = margin_loss(y_true_c, y_pred_c)*LW_C
    ML_m = LW_M*margin_loss(y_true_m, y_pred_m)
    ML_f = LW_F*margin_loss(y_true_f, y_pred_f)
    batch_loss = ML_c + ML_m+ ML_f
    return tf.reduce_mean(batch_loss)

## Model Architecture

In [10]:
def get_compiled_model():
    input_shape = dataset['x_train'].shape[1:]

    input_shape_yc = dataset['y_train_coarse'].shape[1:]
    input_shape_ym = dataset['y_train_medium'].shape[1:]
    input_shape_yf = dataset['y_train_fine'].shape[1:]

    no_coarse_class = number_of_classes_c
    no_medium_class = number_of_classes_m
    no_fine_class = number_of_classes_f

    PCap_n_dims = 16

    SCap_f_dims = 8
    SCap_m_dims = 16
    SCap_c_dims = 32


    # Input image
    x_input = keras.layers.Input(shape=input_shape, name="Input_Image")

    # Input True Labels
    y_c = keras.layers.Input(shape=input_shape_yc, name='input_yc')
    y_m = keras.layers.Input(shape=input_shape_ym, name='input_ym')
    y_f = keras.layers.Input(shape=input_shape_yf, name='input_yf')

    #--- block 1 ---
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(x_input)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    #--- block 2 ---
    x = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    #--- block 3 ---
    x = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    #--- block 4 ---
    x = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)


    # Layer 3: Reshape to 8D primary capsules 
    reshapec = keras.layers.Reshape((int((tf.reduce_prod(x.shape[1:]).numpy())/PCap_n_dims),
                                     PCap_n_dims), name="reshape_layer")(x)
    p_caps = keras.layers.Lambda(models.squash, name='p_caps')(reshapec)

    ## Layer Secondary Capsule: For coarse level
    s_caps_c = models.SecondaryCapsule(n_caps=no_coarse_class, n_dims=SCap_c_dims, 
                        name="s_caps_coarse")(p_caps)
    
    ## Layer Secondary Capsule: For medium level
    s_caps_m = models.SecondaryCapsule(n_caps=no_medium_class, n_dims=SCap_m_dims, 
                        name="s_caps_medium")(s_caps_c)

    ## Layer Secondary Capsule: For fine level
    s_caps_f = models.SecondaryCapsule(n_caps=no_fine_class, n_dims=SCap_f_dims, 
                        name="s_caps_fine")(s_caps_m)

    pred_c = models.LengthLayer(name='prediction_coarse')(s_caps_c)

    pred_m = models.LengthLayer(name='prediction_medium')(s_caps_m)

    pred_f = models.LengthLayer(name='prediction_fine')(s_caps_f)

    model = keras.Model(inputs= [x_input, y_c, y_m, y_f],
                        outputs= [pred_c, pred_m, pred_f],
                        name='HD-CapsNet')

    model.add_loss(CustomLoss(y_c, y_m, y_f, pred_c, pred_m, pred_f, 
                              lossweight['coarse_lw'], lossweight['medium_lw'], lossweight['fine_lw']))

    model.compile(optimizer='adam',                  
                  metrics={'prediction_fine': 'accuracy',
                           'prediction_medium': 'accuracy',
                           'prediction_coarse': 'accuracy'
                          }
    )
    return model

In [11]:
model = get_compiled_model()

In [12]:
model.summary()
# keras.utils.plot_model(model, to_file = directory+"/Architecture.png", show_shapes=True)

Model: "HD-CapsNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_Image (InputLayer)       [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 32, 32, 64)   1792        ['Input_Image[0][0]']            
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 64)  256         ['block1_conv1[0][0]']           
 alization)                                                                                       
                                                                                                  
 block1_conv2 (Conv2D)          (None, 32, 32, 64)   36928       ['batch_normalization[0]

                                                                                                  
 tf.math.subtract_3 (TFOpLambda  (None, 20)          0           ['prediction_medium[0][0]']      
 )                                                                                                
                                                                                                  
 tf.math.subtract_4 (TFOpLambda  (None, 20)          0           ['prediction_medium[0][0]']      
 )                                                                                                
                                                                                                  
 tf.math.maximum (TFOpLambda)   (None, 8)            0           ['tf.math.subtract[0][0]']       
                                                                                                  
 tf.math.subtract_2 (TFOpLambda  (None, 8)           0           ['input_yc[0][0]']               
 )        

 da)                                                                                              
                                                                                                  
 tf.math.add_2 (TFOpLambda)     (None, 100)          0           ['tf.math.multiply_8[0][0]',     
                                                                  'tf.math.multiply_10[0][0]']    
                                                                                                  
 tf.math.multiply_3 (TFOpLambda  (None,)             0           ['tf.math.reduce_sum[0][0]']     
 )                                                                                                
                                                                                                  
 tf.math.multiply_7 (TFOpLambda  (None,)             0           ['tf.math.reduce_sum_1[0][0]']   
 )                                                                                                
          

## Data Augmentation

In [13]:
datagen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1)

training_generator = MixUp_add_loss.MixupGenerator_3level(dataset['x_train'],
                                                 dataset['y_train_coarse'], 
                                                 dataset['y_train_medium'],
                                                 dataset['y_train_fine'],
                                                 batch_size=train_params["batch_size"],
                                                 alpha=0.2, 
                                                 datagen=datagen
                                                )()

## Callback

In [14]:
tb = keras.callbacks.TensorBoard(directory+'./tb_logs'+ datetime.now().strftime("%Y%m%d-%H%M%S"))
log = keras.callbacks.CSVLogger(directory+'/log.csv', append=True)

checkpoint = keras.callbacks.ModelCheckpoint(
    directory+'/epoch-best.h5', 
    monitor='val_prediction_fine_accuracy',
    save_best_only=True, save_weights_only=True, verbose=1)
change_lw = models.LossWeightsModifier(lossweight = lossweight,
                               initial_lw = initial_lw,
                               directory = directory)
lr_decay = keras.callbacks.LearningRateScheduler(scheduler)

## Model Training

In [ ]:
model_save_dir = str(directory+'/trained_model.h5')
try:
    model.load_weights(model_save_dir)
except:
    history = model.fit(training_generator,
                        steps_per_epoch = int(dataset['x_train'].shape[0] / train_params["batch_size"]),
                        epochs = train_params["n_epochs"],
                        validation_data = ([dataset['x_test'],
                                            dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']],
                                           [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]),
                        callbacks = [tb,log,change_lw,lr_decay,checkpoint],
                        verbose=1)
    
    model.save_weights(model_save_dir)

Epoch 1/100
781/781 [==============================] - ETA: 0s - loss: 0.5161 - prediction_coarse_accuracy: 0.4782 - prediction_medium_accuracy: 0.2266 - prediction_fine_accuracy: 0.0126
  • Coarse Accuracy = 47.82% | Val_Accuracy = 52.24% | LossWeight = 0.36 
  • Medium Accuracy = 22.66% | Val_Accuracy = 31.02% | LossWeight = 0.48 
  • Fine   Accuracy = 1.26% | Val_Accuracy = 1.28% | LossWeight = 0.16 

Epoch 1: val_prediction_fine_accuracy improved from -inf to 0.01280, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_WO_C_loss_P16\epoch-best.h5
781/781 [==============================] - 66s 73ms/step - loss: 0.5161 - prediction_coarse_accuracy: 0.4782 - prediction_medium_accuracy: 0.2266 - prediction_fine_accuracy: 0.0126 - val_loss: 0.4468 - val_prediction_coarse_accuracy: 0.5224 - val_prediction_medium_accuracy: 0.3102 - val_prediction_fine_accuracy: 0.0128 - lr: 0.0010
Epoch 2/100
781/781 [==============================] - ETA: 0s - loss: 0.3817 - prediction_coarse_accu

781/781 [==============================] - 69s 88ms/step - loss: 0.2293 - prediction_coarse_accuracy: 0.8426 - prediction_medium_accuracy: 0.7530 - prediction_fine_accuracy: 0.0100 - val_loss: 0.2201 - val_prediction_coarse_accuracy: 0.8078 - val_prediction_medium_accuracy: 0.6976 - val_prediction_fine_accuracy: 0.0100 - lr: 0.0010
Epoch 10/100
781/781 [==============================] - ETA: 0s - loss: 0.2209 - prediction_coarse_accuracy: 0.8551 - prediction_medium_accuracy: 0.7715 - prediction_fine_accuracy: 0.0122
  • Coarse Accuracy = 85.51% | Val_Accuracy = 81.16% | LossWeight = 0.25 
  • Medium Accuracy = 77.15% | Val_Accuracy = 69.42% | LossWeight = 0.35 
  • Fine   Accuracy = 1.22% | Val_Accuracy = 2.09% | LossWeight = 0.40 

Epoch 10: val_prediction_fine_accuracy did not improve from 0.04800
781/781 [==============================] - 70s 89ms/step - loss: 0.2209 - prediction_coarse_accuracy: 0.8551 - prediction_medium_accuracy: 0.7715 - prediction_fine_accuracy: 0.0122 - val_lo

781/781 [==============================] - 69s 88ms/step - loss: 0.1646 - prediction_coarse_accuracy: 0.9192 - prediction_medium_accuracy: 0.8871 - prediction_fine_accuracy: 0.1491 - val_loss: 0.1858 - val_prediction_coarse_accuracy: 0.8374 - val_prediction_medium_accuracy: 0.7353 - val_prediction_fine_accuracy: 0.1438 - lr: 6.6342e-04
Epoch 19/100
781/781 [==============================] - ETA: 0s - loss: 0.1596 - prediction_coarse_accuracy: 0.9243 - prediction_medium_accuracy: 0.8933 - prediction_fine_accuracy: 0.1688
  • Coarse Accuracy = 92.43% | Val_Accuracy = 83.84% | LossWeight = 0.21 
  • Medium Accuracy = 89.33% | Val_Accuracy = 74.02% | LossWeight = 0.26 
  • Fine   Accuracy = 16.88% | Val_Accuracy = 14.98% | LossWeight = 0.53 

Epoch 19: val_prediction_fine_accuracy improved from 0.14380 to 0.14980, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_WO_C_loss_P16\epoch-best.h5
781/781 [==============================] - 62s 80ms/step - loss: 0.1596 - prediction_coarse

781/781 [==============================] - 68s 87ms/step - loss: 0.1373 - prediction_coarse_accuracy: 0.9433 - prediction_medium_accuracy: 0.9248 - prediction_fine_accuracy: 0.2636 - val_loss: 0.1693 - val_prediction_coarse_accuracy: 0.8464 - val_prediction_medium_accuracy: 0.7558 - val_prediction_fine_accuracy: 0.2333 - lr: 4.1812e-04
Epoch 28/100
781/781 [==============================] - ETA: 0s - loss: 0.1366 - prediction_coarse_accuracy: 0.9447 - prediction_medium_accuracy: 0.9280 - prediction_fine_accuracy: 0.2722
  • Coarse Accuracy = 94.47% | Val_Accuracy = 85.01% | LossWeight = 0.19 
  • Medium Accuracy = 92.80% | Val_Accuracy = 75.53% | LossWeight = 0.22 
  • Fine   Accuracy = 27.22% | Val_Accuracy = 23.50% | LossWeight = 0.59 

Epoch 28: val_prediction_fine_accuracy improved from 0.23330 to 0.23500, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_WO_C_loss_P16\epoch-best.h5
781/781 [==============================] - 71s 91ms/step - loss: 0.1366 - prediction_coarse

781/781 [==============================] - 67s 86ms/step - loss: 0.1277 - prediction_coarse_accuracy: 0.9501 - prediction_medium_accuracy: 0.9360 - prediction_fine_accuracy: 0.3193 - val_loss: 0.1631 - val_prediction_coarse_accuracy: 0.8535 - val_prediction_medium_accuracy: 0.7605 - val_prediction_fine_accuracy: 0.2706 - lr: 2.6352e-04
Epoch 37/100
781/781 [==============================] - ETA: 0s - loss: 0.1273 - prediction_coarse_accuracy: 0.9513 - prediction_medium_accuracy: 0.9366 - prediction_fine_accuracy: 0.3318
  • Coarse Accuracy = 95.13% | Val_Accuracy = 85.35% | LossWeight = 0.19 
  • Medium Accuracy = 93.66% | Val_Accuracy = 75.94% | LossWeight = 0.22 
  • Fine   Accuracy = 33.18% | Val_Accuracy = 27.90% | LossWeight = 0.60 

Epoch 37: val_prediction_fine_accuracy improved from 0.27060 to 0.27900, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_WO_C_loss_P16\epoch-best.h5
781/781 [==============================] - 64s 82ms/step - loss: 0.1273 - prediction_coarse

Epoch 46/100
781/781 [==============================] - ETA: 0s - loss: 0.1212 - prediction_coarse_accuracy: 0.9552 - prediction_medium_accuracy: 0.9437 - prediction_fine_accuracy: 0.3758
  • Coarse Accuracy = 95.52% | Val_Accuracy = 85.46% | LossWeight = 0.19 
  • Medium Accuracy = 94.37% | Val_Accuracy = 76.59% | LossWeight = 0.21 
  • Fine   Accuracy = 37.58% | Val_Accuracy = 30.92% | LossWeight = 0.60 

Epoch 46: val_prediction_fine_accuracy improved from 0.30510 to 0.30920, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_WO_C_loss_P16\epoch-best.h5
781/781 [==============================] - 63s 80ms/step - loss: 0.1212 - prediction_coarse_accuracy: 0.9552 - prediction_medium_accuracy: 0.9437 - prediction_fine_accuracy: 0.3758 - val_loss: 0.1553 - val_prediction_coarse_accuracy: 0.8546 - val_prediction_medium_accuracy: 0.7659 - val_prediction_fine_accuracy: 0.3092 - lr: 1.5778e-04
Epoch 47/100
781/781 [==============================] - ETA: 0s - loss: 0.1208 - prediction

781/781 [==============================] - 51s 65ms/step - loss: 0.1184 - prediction_coarse_accuracy: 0.9573 - prediction_medium_accuracy: 0.9450 - prediction_fine_accuracy: 0.4063 - val_loss: 0.1527 - val_prediction_coarse_accuracy: 0.8584 - val_prediction_medium_accuracy: 0.7711 - val_prediction_fine_accuracy: 0.3295 - lr: 1.0467e-04
Epoch 55/100
781/781 [==============================] - ETA: 0s - loss: 0.1179 - prediction_coarse_accuracy: 0.9573 - prediction_medium_accuracy: 0.9460 - prediction_fine_accuracy: 0.4072
  • Coarse Accuracy = 95.73% | Val_Accuracy = 85.77% | LossWeight = 0.19 
  • Medium Accuracy = 94.60% | Val_Accuracy = 77.05% | LossWeight = 0.21 
  • Fine   Accuracy = 40.72% | Val_Accuracy = 33.12% | LossWeight = 0.60 

Epoch 55: val_prediction_fine_accuracy improved from 0.32950 to 0.33120, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_WO_C_loss_P16\epoch-best.h5
781/781 [==============================] - 51s 66ms/step - loss: 0.1179 - prediction_coarse

Epoch 64/100
781/781 [==============================] - ETA: 0s - loss: 0.1168 - prediction_coarse_accuracy: 0.9584 - prediction_medium_accuracy: 0.9475 - prediction_fine_accuracy: 0.4231
  • Coarse Accuracy = 95.84% | Val_Accuracy = 85.96% | LossWeight = 0.19 
  • Medium Accuracy = 94.75% | Val_Accuracy = 77.19% | LossWeight = 0.21 
  • Fine   Accuracy = 42.31% | Val_Accuracy = 34.06% | LossWeight = 0.60 

Epoch 64: val_prediction_fine_accuracy did not improve from 0.34240
781/781 [==============================] - 49s 63ms/step - loss: 0.1168 - prediction_coarse_accuracy: 0.9584 - prediction_medium_accuracy: 0.9475 - prediction_fine_accuracy: 0.4231 - val_loss: 0.1510 - val_prediction_coarse_accuracy: 0.8596 - val_prediction_medium_accuracy: 0.7719 - val_prediction_fine_accuracy: 0.3406 - lr: 6.2672e-05
Epoch 65/100
781/781 [==============================] - ETA: 0s - loss: 0.1167 - prediction_coarse_accuracy: 0.9582 - prediction_medium_accuracy: 0.9469 - prediction_fine_accuracy: 0.

781/781 [==============================] - ETA: 0s - loss: 0.1157 - prediction_coarse_accuracy: 0.9568 - prediction_medium_accuracy: 0.9469 - prediction_fine_accuracy: 0.4372
  • Coarse Accuracy = 95.68% | Val_Accuracy = 86.00% | LossWeight = 0.19 
  • Medium Accuracy = 94.69% | Val_Accuracy = 77.35% | LossWeight = 0.21 
  • Fine   Accuracy = 43.72% | Val_Accuracy = 35.35% | LossWeight = 0.59 

Epoch 73: val_prediction_fine_accuracy did not improve from 0.35420
781/781 [==============================] - 76s 98ms/step - loss: 0.1157 - prediction_coarse_accuracy: 0.9568 - prediction_medium_accuracy: 0.9469 - prediction_fine_accuracy: 0.4372 - val_loss: 0.1495 - val_prediction_coarse_accuracy: 0.8600 - val_prediction_medium_accuracy: 0.7735 - val_prediction_fine_accuracy: 0.3535 - lr: 3.9499e-05
Epoch 74/100
781/781 [==============================] - ETA: 0s - loss: 0.1165 - prediction_coarse_accuracy: 0.9584 - prediction_medium_accuracy: 0.9481 - prediction_fine_accuracy: 0.4328
  • Coar

781/781 [==============================] - ETA: 0s - loss: 0.1156 - prediction_coarse_accuracy: 0.9568 - prediction_medium_accuracy: 0.9460 - prediction_fine_accuracy: 0.4421
  • Coarse Accuracy = 95.68% | Val_Accuracy = 86.11% | LossWeight = 0.19 
  • Medium Accuracy = 94.60% | Val_Accuracy = 77.56% | LossWeight = 0.22 
  • Fine   Accuracy = 44.21% | Val_Accuracy = 35.98% | LossWeight = 0.59 

Epoch 82: val_prediction_fine_accuracy did not improve from 0.36030
781/781 [==============================] - 84s 108ms/step - loss: 0.1156 - prediction_coarse_accuracy: 0.9568 - prediction_medium_accuracy: 0.9460 - prediction_fine_accuracy: 0.4421 - val_loss: 0.1491 - val_prediction_coarse_accuracy: 0.8611 - val_prediction_medium_accuracy: 0.7756 - val_prediction_fine_accuracy: 0.3598 - lr: 2.4894e-05
Epoch 83/100
781/781 [==============================] - ETA: 0s - loss: 0.1159 - prediction_coarse_accuracy: 0.9578 - prediction_medium_accuracy: 0.9487 - prediction_fine_accuracy: 0.4439
  • Coa

In [ ]:
try:
    history_dict = history.history

    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({"Coarse": history}, metric = "prediction_coarse_accuracy")
    plotter.plot({"Medium": history}, metric = "prediction_medium_accuracy")
    plotter.plot({"Fine": history}, metric = "prediction_fine_accuracy")
    plt.title("Model Accuracy")
    plt.ylim([0,1])
except:
    print('Trained model weights loaded')

In [ ]:
try:
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({"loss": history}, metric = "loss")
    plt.title("Model Loss")
    plt.ylim([0,1])
except:
    print('Trained model weights loaded')

## Model Analysis

In [ ]:
final_model = keras.Model(model.inputs[:1], model.output)

lossfn = models.MarginLoss()
final_model.compile(optimizer='adam', 
                    loss={'prediction_fine' : lossfn,
                          'prediction_medium' : lossfn,
                          'prediction_coarse' : lossfn},

                    loss_weights={'prediction_fine' : lossweight['fine_lw'],
                                  'prediction_medium' : lossweight['medium_lw'],
                                  'prediction_coarse' : lossweight['coarse_lw']},

                    metrics={'prediction_fine': 'accuracy',
                             'prediction_medium': 'accuracy',
                             'prediction_coarse': 'accuracy'
                            }
                   )

In [ ]:
model_analysis = models.model_analysis(final_model, dataset)
results = model_analysis.evaluate()
predictions = model_analysis.prediction()

In [ ]:
true_label = [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]
pred_label = [predictions[0],predictions[1],predictions[2]]
metrics.lvl_wise_metric(true_label,pred_label)

In [ ]:
h_measurements,consistency,exact_match = metrics.hmeasurements(true_label,
                                       pred_label,
                                       dataset['tree'])
print('\nHierarchical Precision =',h_measurements[0],
      '\nHierarchical Recall =', h_measurements[1],
      '\nHierarchical F1-Score =',h_measurements[2],
      '\nConsistency = ', consistency,
      '\nExact Match = ', exact_match,
     )

# Training Again

In [ ]:
model_save_dir = str(directory+'/trained_model_2.h5')
try:
    model.load_weights(model_save_dir)
except:
    history = model.fit(training_generator,
                        steps_per_epoch = int(dataset['x_train'].shape[0] / train_params["batch_size"]),
                        epochs = train_params["n_epochs"],
                        validation_data = ([dataset['x_test'],
                                            dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']],
                                           [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]),
                        callbacks = [tb,log,change_lw,lr_decay,checkpoint],
                        verbose=1)
    
    model.save_weights(model_save_dir)

In [ ]:
history_dict = history.history

In [ ]:
plotter = tfdocs.plots.HistoryPlotter()
plotter.plot({"Coarse": history}, metric = "prediction_coarse_accuracy")
plotter.plot({"Medium": history}, metric = "prediction_medium_accuracy")
plotter.plot({"Fine": history}, metric = "prediction_fine_accuracy")
plt.title("Model Accuracy")
plt.ylim([0,1])

In [ ]:
plotter = tfdocs.plots.HistoryPlotter()
plotter.plot({"loss": history}, metric = "loss")
plt.title("Model Loss")
plt.ylim([0,1])

## Model Analysis

In [ ]:
final_model = keras.Model(model.inputs[:1], model.output)

lossfn = models.MarginLoss()
final_model.compile(optimizer='adam', 
                    loss={'prediction_fine' : lossfn,
                          'prediction_medium' : lossfn,
                          'prediction_coarse' : lossfn},

                    loss_weights={'prediction_fine' : lossweight['fine_lw'],
                                  'prediction_medium' : lossweight['medium_lw'],
                                  'prediction_coarse' : lossweight['coarse_lw']},

                    metrics={'prediction_fine': 'accuracy',
                             'prediction_medium': 'accuracy',
                             'prediction_coarse': 'accuracy'
                            }
                   )

In [ ]:
model_analysis = models.model_analysis(final_model, dataset)
results = model_analysis.evaluate()
predictions = model_analysis.prediction()

In [ ]:
true_label = [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]
pred_label = [predictions[0],predictions[1],predictions[2]]
metrics.lvl_wise_metric(true_label,pred_label)

In [ ]:
h_measurements,consistency,exact_match = metrics.hmeasurements(true_label,
                                       pred_label,
                                       dataset['tree'])
print('\nHierarchical Precision =',h_measurements[0],
      '\nHierarchical Recall =', h_measurements[1],
      '\nHierarchical F1-Score =',h_measurements[2],
      '\nConsistency = ', consistency,
      '\nExact Match = ', exact_match,
     )